In [ ]:
import os
from functools import partial
import yaml
import hashlib
from io import BytesIO
from zipfile import ZipFile

from IPython import display
import ipywidgets
from ipywidgets import Label, Dropdown, HBox, VBox, Text, Button, Valid
import traitlets
from traitlets import Unicode
from tornado.httpclient import AsyncHTTPClient

import db
db.use_db()

## File Upload Widget

In [ ]:
class FileWidget(ipywidgets.DOMWidget):
    _view_name = Unicode('FilePickerView').tag(sync=True)
    _view_module = Unicode('filepicker').tag(sync=True)

    value = Unicode().tag(sync=True)
    filename = Unicode().tag(sync=True)

    def __init__(self, **kwargs):
        """Constructor"""
        ipywidgets.DOMWidget.__init__(self, **kwargs) # Call the base.

        # Allow the user to register error callbacks with the following signatures:
        #    callback()
        #    callback(sender)
        self.errors = ipywidgets.CallbackDispatcher(accepted_nargs=[0, 1])

        # Listen for custom msgs
        self.on_msg(self._handle_custom_msg)

    def _handle_custom_msg(self, content):
        """Handle a msg from the front-end.

        Parameters
        ----------
        content: dict
            Content of the msg."""
        if 'event' in content and content['event'] == 'error':
            self.errors()
            self.errors(self)

In [ ]:
%%javascript

requirejs.undef('filepicker');

define('filepicker', ["jupyter-js-widgets"], function(widgets) {

    var FilePickerView = widgets.DOMWidgetView.extend({

        render: function() {
            // Render the view.
            this.setElement($('<input />')
                .attr('type', 'file'));
        },

        events: {
            // List of events and their handlers.
            'change': 'handle_file_change',
        },

        handle_file_change: function(evt) {
            // Handle when the user has changed the file.

            // Retrieve the first (and only!) File from the FileList object
            var file = evt.target.files[0];
            if (! file) { 
                // The file couldn't be opened.  Send an error msg to the
                // back-end.
                this.send({ 'event': 'error' });
                return ;
            }
            
            // check if has 'yml' extension

            // Read the file's textual content and set value to those contents.
            var that = this;
            var file_reader = new FileReader();
            file_reader.onload = function(e) {
                that.model.set('value', e.target.result);
                that.touch();
                console.log('sent from client side')
            }
            file_reader.readAsText(file);

            // Set the filename of the file.
            this.model.set('filename', file.name);
            this.touch();
        },
    });

    return {
        FilePickerView: FilePickerView
    };
});

---
## Utilities

In [ ]:
def get_exam_names():
    with db.db_session:
        return list(db.orm.select(ex.name for ex in db.Exam))
    
def get_exam_data(exam_name):
    if not exam_name:
        return None
    with db.db_session:
        exam = db.Exam.get(name=exam_name)
        if not exam:
            return None
        yaml_path = exam.yaml_path
    try:
        return db.read_yaml(yaml_path)
    except Exception:
        return None

def observe(widget, **kwargs):
    return partial(widget.observe, **kwargs)

---
## Widget Init

In [ ]:
yaml_upload_label = Label("Upload YAML", margin="0 10px 0 0")
yaml_upload = FileWidget()
yaml_upload_valid = Valid()
yaml_upload_status = Label()
yaml_upload_box = VBox([HBox([yaml_upload_label, yaml_upload, ]),
                        HBox([yaml_upload_status, yaml_upload_valid])])

yaml_upload_valid.layout.display = 'none'
yaml_upload_valid.layout.margin = "0 10px"

# Register an event to echo the filename when it has been changed.
@observe(yaml_upload, names="filename")
def file_loading(change):
    yaml_upload.value = ''
    yaml_upload_valid.layout.display = 'none'
    yaml_upload_status.value = "Loading " + yaml_upload.filename

# Register an event to echo the filename and contents when a file
# has been uploaded.
@observe(yaml_upload, names="value")
def file_loaded(change):
    if not yaml_upload.value:
        return
    # validate yaml
    yaml_upload_status.value = "Validating " + yaml_upload.filename
    try:
        yml = yaml.load(yaml_upload.value)
        
        try:
            yml = db.clean_yaml(yml)  # attempt to clean
        except Exception:
            pass  # if the YAML is not valid parsing will raise anyway
        exam_name, qr, widgets = db.parse_yaml(yml)
        if select_exam.value and select_exam.value != exam_name:
            raise ValueError('Exam in YAML file does not match selected exam')
        # If there is already yaml for this exam, load it now so we can
        # compute a diff later
        existing_exam_data = get_exam_data(exam_name)
        if existing_exam_data:
            raise ValueError('Exam data for {} already exists'.format(exam_name))
    except ValueError as exc:
        yaml_upload_valid.value = False
        yaml_upload_valid.layout.display = 'inline'
        yaml_upload_status.value = str(exc)
        return
    except Exception as exc:
        yaml_upload_valid.value = False
        yaml_upload_valid.layout.display = 'inline'
        yaml_upload_status.value = "Invalid YAML in " + yaml_upload.filename
        return

    # save yaml
    yaml_upload_status.value = "Saving " + yaml_upload.filename
    try:
        with open(yaml_upload.filename, 'w') as f:
            f.write(yaml.dump(yml))
    except Exception:
        yaml_upload_valid.value = False
        yaml_upload_valid.layout.display = 'inline'
        yaml_upload_status.value = "Failed to save " + yaml_upload.filename
        return
    

    yaml_upload_status.value = "Adding exam to database"
    
    try:
        db.add_exam(yaml_upload.filename)
        os.makedirs(exam_name + '_data', exist_ok=True)
        yaml_upload_status.value = "Added exam to database"
    except Exception as exc:
        yaml_upload_valid.value = False
        yaml_upload_valid.layout.display = 'inline'
        yaml_upload_status.value = "Failed to add exam to database"
        return
    finally:
        # XXX: use of global variables
        # update list of exams
        select_exam.options = [None] + get_exam_names()
    

    if existing_exam_data:
        yaml_upload_status.value = "Updating question names"
        # always do this, as is quick
        # calculate diff between YAMLs
        if True:  # widget coordinates changed
            yaml_upload_status.value = "Modifying widget coordinates"
            # grab page images from database and re-run widget extraction


    yaml_upload_status.value = "YAML imported successfully"
    yaml_upload_valid.value = True
    yaml_upload_valid.layout.display = 'inline'

# Register an event to print an error message when a file could not
# be opened.  Since the error messages are not handled through
# traitlets but instead handled through custom msgs, the registration
# of the handler is different than the two examples above.  Instead
# the API provided by the CallbackDispatcher must be used.
def file_failed():
    yaml_upload_valid.value = False
    yaml_upload_valid.layout.display = 'inline'
    yaml_upload_status.value = 'Could not load file contents of ' + yaml_upload.filename
yaml_upload.errors.register_callback(file_failed)

In [ ]:
pdf_label = Label("Exam PDF URL", margin="7px 10px")
pdf_url = Text()
pdf_import = Button(description="Import exam data")

pdf_import_status = Label()
pdf_import_valid = Valid()
pdf_import_valid.layout.display = 'none'
pdf_import_valid.layout.margin = "0 10px"

pdf_import_box = VBox([HBox([pdf_label ,pdf_url, pdf_import]),
                       HBox([pdf_import_status, pdf_import_valid])])
pdf_import_box.layout.display = 'none'  # initially no exam is selected, so we hide this

# this is global state to prevent the UI triggering another download
# while there is one in progress
pdf_download_client = None


def get_new_client():
    # We should probably use a streaming client, but for simplicity we just set the max buffer size to 500MB
    return AsyncHTTPClient(max_buffer_size=int(500E8))


def pdf_name(pdf_content):
    return hashlib.sha256(pdf_content).hexdigest()[:15] + '.pdf'


def handle_pdf(pdf, name='PDF'):
    exam_dir = select_exam.value + '_data'
    filename = os.path.join(exam_dir, pdf_name(pdf))
    pdf_import_status.value = 'Saving {}'.format(name)
    try:
        with open(filename, 'wb') as f:
            f.write(pdf)
    except Exception as e:
        raise RuntimeError('Failed to save {}'.format(name))

    pdf_import_status.value = 'Importing {}'.format(name)
    try:
        db.process_pdf(filename, select_exam.value + '.yml')
    except Exception:
        raise RuntimeError('Failed to import {}'.format(name))

    pdf_import_valid.value = True
    pdf_import_valid.layout.display = 'inline'
    pdf_import_status.value = 'Imported {}'.format(name)

def handle_zip(zipcontents):
    exam_dir = select_exam.value + '_data'
    pdf_import_status.value = 'Extracting PDFs from ZIP archive'
    with ZipFile(BytesIO(zipcontents)) as zf:
        pdf_files = list(filter(lambda n: n.endswith('.pdf'), zf.namelist()))
        for pdf in pdf_files:
            pdf_import_status.value = 'Extracting {}'.format(pdf)
            handle_pdf(zf.open(pdf).read(), name=pdf)
            pdf_import_valid.layout.display = 'none'  # set at the end of handle_pdf
    
    pdf_import_valid.value = True
    pdf_import_valid.layout.display = 'inline'
    pdf_import_status.value = 'Imported: {}'.format(', '.join(pdf_files))        


content_type_router = {
    'application/pdf': handle_pdf,
    'application/zip': handle_zip,
}

def handle_response(response):    
    global rsp, pdf_download_client
    rsp = response
    try:
        if response.error:
            raise RuntimeError('Failed to fetch file(s)', response.error)
        try:
            handler = content_type_router[response.headers['Content-Type']]
            handler(response.body)
        except KeyError:
            raise RuntimeError('Link does not point to a PDF/ZIP file')
    except Exception as exc:
        pdf_import_valid.value = False
        pdf_import_valid.layout.display = 'inline'
        pdf_import_status.value = str(exc)
    finally:
        pdf_download_client = None  # allow next upload



@pdf_import.on_click
def _(sender):
    global pdf_download_client
    if pdf_download_client:
        return  # download already underway
    pdf_download_client = get_new_client()
    # 30 second timeout
    pdf_download_client.fetch(pdf_url.value.strip(), handle_response, request_timeout=30)
    pdf_import_valid.layout.display = 'none'
    pdf_import_status.value = 'Fetching file'


In [ ]:
select_exam = Dropdown(description='Exam', options= [None] + get_exam_names())

@observe(select_exam, names="value")
def _(state):
    display = {False: 'none', True: 'inline'}
    value = state['new']
    pdf_import_box.layout.display = display[value is not None]

---
## UI Layout

In [ ]:
select_exam

In [ ]:
VBox([yaml_upload_box, pdf_import_box])